# Run model module on GCP with unlabeled threshold tuning

In [ ]:
import os
PROJECT = "PROJECT" # REPLACE WITH YOUR PROJECT ID
BUCKET = "BUCKET" # REPLACE WITH A BUCKET NAME
REGION = "us-central1" # REPLACE WITH YOUR REGION e.g. us-central1

# Import os environment variables
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] =  BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "1.13"

Copy data over to bucket

In [ ]:
%%bash
gsutil -m cp -r data/* gs://${BUCKET}/anomaly_detection/data

In [ ]:
# Import os environment variables for global sequence shape hyperparameters
os.environ["SEQ_LEN"] = str(30)
os.environ["NUM_FEAT"] = str(5)

# Import os environment variables for global feature hyperparameters
os.environ["FEAT_NAMES"] = (",").join(["tag_{}".format(i) for i in range(int(os.environ["NUM_FEAT"]))])
os.environ["FEAT_DEFAULTS"] = (",").join([(";").join(["0.0"] * int(os.environ["SEQ_LEN"]))] * int(os.environ["NUM_FEAT"]))

# Import os environment variables for global training hyperparameters
os.environ["START_DELAY_SECS"] = str(60)
os.environ["THROTTLE_SECS"] = str(120)

# Import os environment variables for global threshold hyperparameters
os.environ["LABELED_TUNE_THRESH"] = "False"

# Import global dense hyperparameters
os.environ["ENC_DNN_HIDDEN_UNITS"] = "64,32,16"
os.environ["LATENT_VECTOR_SIZE"] = str(8)
os.environ["DEC_DNN_HIDDEN_UNITS"] = "16,32,64"
os.environ["TIME_LOSS_WEIGHT"] = str(1.0)
os.environ["FEAT_LOSS_WEIGHT"] = str(1.0)

# Import global lstm hyperparameters
os.environ["REVERSE_LABELS_SEQUENCE"] = "True"
os.environ["ENC_LSTM_HIDDEN_UNITS"] = "64,32,16"
os.environ["DEC_LSTM_HIDDEN_UNITS"] = "16,32,64"
os.environ["LSTM_DROPOUT_OUTPUT_KEEP_PROBS"] = "0.9,0.95,1.0"
os.environ["DNN_HIDDEN_UNITS"] = "1024,256,64"

## Train reconstruction variables

In [ ]:
# Import os environment variables for reconstruction training hyperparameters
os.environ["TRAIN_FILE_PATTERN"] = "gs://{}/anomaly_detection/data/train_norm_seq.csv".format(BUCKET)
os.environ["EVAL_FILE_PATTERN"] = "gs://{}/anomaly_detection/data/val_norm_1_seq.csv".format(BUCKET)
os.environ["PREVIOUS_TRAIN_STEPS"] = str(0)
os.environ["RECONSTRUCTION_EPOCHS"] = str(1.0)
os.environ["TRAIN_EXAMPLES"] = str(64000)
os.environ["LEARNING_RATE"] = str(0.1)
os.environ["TRAINING_MODE"] = "reconstruction"

### Dense Autoencoder

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/anomaly_detection/trained_model/dense_unlabeled
JOBNAME=job_anomaly_detection_reconstruction_dense_unlabeled_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBNAME}
gsutil -m rm -rf ${OUTDIR}
gcloud ml-engine jobs submit training ${JOBNAME} \
  --region=${REGION} \
  --module-name=trainer.task \
  --package-path=$PWD/anomaly_detection_module/trainer \
  --job-dir=${OUTDIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=STANDARD_1 \
  --runtime-version=${TFVERSION} \
  -- \
  --train_file_pattern=${TRAIN_FILE_PATTERN} \
  --eval_file_pattern=${EVAL_FILE_PATTERN} \
  --output_dir=${OUTDIR} \
  --job-dir=./tmp \
  --seq_len=${SEQ_LEN} \
  --num_feat=${NUM_FEAT} \
  --feat_names=${FEAT_NAMES} \
  --feat_defaults=${FEAT_DEFAULTS} \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --previous_train_steps=${PREVIOUS_TRAIN_STEPS} \
  --reconstruction_epochs=${RECONSTRUCTION_EPOCHS} \
  --train_examples=${TRAIN_EXAMPLES} \
  --learning_rate=${LEARNING_RATE} \
  --start_delay_secs=${START_DELAY_SECS} \
  --throttle_secs=${THROTTLE_SECS} \
  --model_type="dense_autoencoder" \
  --enc_dnn_hidden_units=${ENC_DNN_HIDDEN_UNITS} \
  --latent_vector_size=${LATENT_VECTOR_SIZE} \
  --dec_dnn_hidden_units=${DEC_DNN_HIDDEN_UNITS} \
  --time_loss_weight=${TIME_LOSS_WEIGHT} \
  --feat_loss_weight=${FEAT_LOSS_WEIGHT} \
  --training_mode=${TRAINING_MODE} \
  --labeled_tune_thresh=${LABELED_TUNE_THRESH}

### LSTM Autoencoder

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/anomaly_detection/trained_model/lstm_unlabeled
JOBNAME=job_anomaly_detection_reconstruction_lstm_unlabeled_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBNAME}
gsutil -m rm -rf ${OUTDIR}
gcloud ml-engine jobs submit training ${JOBNAME} \
  --region=${REGION} \
  --module-name=trainer.task \
  --package-path=$PWD/anomaly_detection_module/trainer \
  --job-dir=${OUTDIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=STANDARD_1 \
  --runtime-version=${TFVERSION} \
  -- \
  --train_file_pattern=${TRAIN_FILE_PATTERN} \
  --eval_file_pattern=${EVAL_FILE_PATTERN} \
  --output_dir=${OUTDIR} \
  --job-dir=./tmp \
  --seq_len=${SEQ_LEN} \
  --num_feat=${NUM_FEAT} \
  --feat_names=${FEAT_NAMES} \
  --feat_defaults=${FEAT_DEFAULTS} \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --previous_train_steps=${PREVIOUS_TRAIN_STEPS} \
  --reconstruction_epochs=${RECONSTRUCTION_EPOCHS} \
  --train_examples=${TRAIN_EXAMPLES} \
  --learning_rate=${LEARNING_RATE} \
  --start_delay_secs=${START_DELAY_SECS} \
  --throttle_secs=${THROTTLE_SECS} \
  --model_type="lstm_enc_dec_autoencoder" \
  --reverse_labels_sequence=${REVERSE_LABELS_SEQUENCE} \
  --enc_lstm_hidden_units=${ENC_LSTM_HIDDEN_UNITS} \
  --dec_lstm_hidden_units=${DEC_LSTM_HIDDEN_UNITS} \
  --lstm_dropout_output_keep_probs=${LSTM_DROPOUT_OUTPUT_KEEP_PROBS} \
  --dnn_hidden_units=${DNN_HIDDEN_UNITS} \
  --training_mode=${TRAINING_MODE} \
  --labeled_tune_thresh=${LABELED_TUNE_THRESH}

### PCA Autoencoder

Reconstruction

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/anomaly_detection/trained_model/pca_unlabeled
JOBNAME=job_anomaly_detection_reconstruction_pca_unlabeled_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBNAME}
gsutil -m rm -rf ${OUTDIR}
gcloud ml-engine jobs submit training ${JOBNAME} \
  --region=${REGION} \
  --module-name=trainer.task \
  --package-path=$PWD/anomaly_detection_module/trainer \
  --job-dir=${OUTDIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=STANDARD_1 \
  --runtime-version=${TFVERSION} \
  -- \
  --train_file_pattern=${TRAIN_FILE_PATTERN} \
  --eval_file_pattern=${EVAL_FILE_PATTERN} \
  --output_dir=${OUTDIR} \
  --job-dir=./tmp \
  --seq_len=${SEQ_LEN} \
  --num_feat=${NUM_FEAT} \
  --feat_names=${FEAT_NAMES} \
  --feat_defaults=${FEAT_DEFAULTS} \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --previous_train_steps=${PREVIOUS_TRAIN_STEPS} \
  --reconstruction_epochs=1.0 \
  --train_examples=${TRAIN_EXAMPLES} \
  --eval_examples=6400 \
  --start_delay_secs=${START_DELAY_SECS} \
  --throttle_secs=${THROTTLE_SECS} \
  --model_type="pca" \
  --training_mode=${TRAINING_MODE} \
  --autotune_principal_components="False" \
  --labeled_tune_thresh=${LABELED_TUNE_THRESH}

Autotune principal components

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/anomaly_detection/trained_model/pca_unlabeled
JOBNAME=job_anomaly_detection_reconstruction_pca_unlabeled_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBNAME}
gcloud ml-engine jobs submit training ${JOBNAME} \
  --region=${REGION} \
  --module-name=trainer.task \
  --package-path=$PWD/anomaly_detection_module/trainer \
  --job-dir=${OUTDIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=STANDARD_1 \
  --runtime-version=${TFVERSION} \
  -- \
  --train_file_pattern=${EVAL_FILE_PATTERN} \
  --eval_file_pattern=${EVAL_FILE_PATTERN} \
  --output_dir=${OUTDIR} \
  --job-dir=./tmp \
  --seq_len=${SEQ_LEN} \
  --num_feat=${NUM_FEAT} \
  --feat_names=${FEAT_NAMES} \
  --feat_defaults=${FEAT_DEFAULTS} \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --previous_train_steps=2000 \
  --reconstruction_epochs=1.0 \
  --train_examples=6400 \
  --eval_examples=6400 \
  --start_delay_secs=${START_DELAY_SECS} \
  --throttle_secs=${THROTTLE_SECS} \
  --model_type="pca" \
  --training_mode=${TRAINING_MODE} \
  --autotune_principal_components="True" \
  --labeled_tune_thresh=${LABELED_TUNE_THRESH}

## Hyperparameter tuning of reconstruction hyperparameters

### Dense Autoencoder

In [ ]:
%%writefile hyperparam_reconstruction_dense.yaml
trainingInput:
  scaleTier: STANDARD_1
  hyperparameters:
    hyperparameterMetricTag: rmse
    goal: MINIMIZE
    maxTrials: 30
    maxParallelTrials: 1
    params:
    - parameterName: enc_dnn_hidden_units
      type: CATEGORICAL
      categoricalValues: ["64 32 16", "256 128 16", "64 64 64"]
    - parameterName: latent_vector_size
      type: INTEGER
      minValue: 8
      maxValue: 16
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: dec_dnn_hidden_units
      type: CATEGORICAL
      categoricalValues: ["16 32 64", "16 128 256", "64 64 64"]
    - parameterName: train_batch_size
      type: INTEGER
      minValue: 8
      maxValue: 512
      scaleType: UNIT_LOG_SCALE
    - parameterName: learning_rate
      type: DOUBLE
      minValue: 0.001
      maxValue: 0.1
      scaleType: UNIT_LINEAR_SCALE

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/anomaly_detection/hyperparam_reconstruction_dense_unlabeled
JOBNAME=job_anomaly_detection_hyperparam_reconstruction_dense_unlabeled_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBNAME}
gsutil -m rm -rf ${OUTDIR}
gcloud ml-engine jobs submit training ${JOBNAME} \
  --region=${REGION} \
  --module-name=trainer.task \
  --package-path=$PWD/anomaly_detection_module/trainer \
  --job-dir=${OUTDIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=STANDARD_1 \
  --config=hyperparam_reconstruction_dense.yaml \
  --runtime-version=${TFVERSION} \
  -- \
  --train_file_pattern=gs://${BUCKET}/anomaly_detection/data/train_norm_seq.csv \
  --eval_file_pattern=gs://${BUCKET}/anomaly_detection/data/val_norm_1_seq.csv \
  --output_dir=${OUTDIR} \
  --job-dir=${OUTDIR} \
  --seq_len=30 \
  --num_feat=5 \
  --feat_names=${FEAT_NAMES} \
  --feat_defaults=${FEAT_DEFAULTS} \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --previous_train_steps=0 \
  --reconstruction_epochs=1.0 \
  --train_examples=64000 \
  --start_delay_secs=60 \
  --throttle_secs=120 \
  --training_mode="reconstruction" \
  --labeled_tune_thresh=True \
  --num_time_anom_thresh=300 \
  --num_feat_anom_thresh=300

### LSTM Autoencoder

In [ ]:
%%writefile hyperparam_reconstruction_lstm.yaml
trainingInput:
  scaleTier: STANDARD_1
  hyperparameters:
    hyperparameterMetricTag: rmse
    goal: MINIMIZE
    maxTrials: 30
    maxParallelTrials: 1
    params:
    - parameterName: enc_lstm_hidden_units
      type: CATEGORICAL
      categoricalValues: ["64 32 16", "256 128 16", "64 64 64"]
    - parameterName: dec_lstm_hidden_units
      type: CATEGORICAL
      categoricalValues: ["16 32 64", "16 128 256", "64 64 64"]
    - parameterName: lstm_dropout_output_keep_probs
      type: CATEGORICAL
      categoricalValues: ["0.9 1.0 1.0", "0.95 0.95 1.0", "0.95 0.95 0.95"]
    - parameterName: dnn_hidden_units
      type: CATEGORICAL
      categoricalValues: ["256 128 64", "256 128 16", "64 64 64"]
    - parameterName: train_batch_size
      type: INTEGER
      minValue: 8
      maxValue: 512
      scaleType: UNIT_LOG_SCALE
    - parameterName: learning_rate
      type: DOUBLE
      minValue: 0.001
      maxValue: 0.1
      scaleType: UNIT_LINEAR_SCALE

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/anomaly_detection/hyperparam_reconstruction_lstm_unlabeled
JOBNAME=job_anomaly_detection_hyperparam_reconstruction_lstm_unlabeled_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBNAME}
gsutil -m rm -rf ${OUTDIR}
gcloud ml-engine jobs submit training ${JOBNAME} \
  --region=${REGION} \
  --module-name=trainer.task \
  --package-path=$PWD/anomaly_detection_module/trainer \
  --job-dir=${OUTDIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=STANDARD_1 \
  --config=hyperparam_reconstruction_lstm.yaml \
  --runtime-version=${TFVERSION} \
  -- \
  --train_file_pattern=gs://${BUCKET}/anomaly_detection/data/train_norm_seq.csv \
  --eval_file_pattern=gs://${BUCKET}/anomaly_detection/data/val_norm_1_seq.csv \
  --output_dir=${OUTDIR} \
  --job-dir=${OUTDIR} \
  --seq_len=30 \
  --num_feat=5 \
  --feat_names=${FEAT_NAMES} \
  --feat_defaults=${FEAT_DEFAULTS} \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --previous_train_steps=0 \
  --reconstruction_epochs=1.0 \
  --train_examples=64000 \
  --start_delay_secs=60 \
  --throttle_secs=120 \
  --training_mode="reconstruction" \
  --labeled_tune_thresh=True \
  --num_time_anom_thresh=300 \
  --num_feat_anom_thresh=300

### PCA Autoencoder

In [ ]:
%%writefile hyperparam_reconstruction_pca.yaml
trainingInput:
  scaleTier: STANDARD_1
  hyperparameters:
    hyperparameterMetricTag: rmse
    goal: MINIMIZE
    maxTrials: 30
    maxParallelTrials: 1
    params:
    - parameterName: k_principal_components_time
      type: INTEGER
      minValue: 2
      maxValue: 10
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: k_principal_components_feat
      type: INTEGER
      minValue: 2
      maxValue: 10
      scaleType: UNIT_LINEAR_SCALE

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/anomaly_detection/hyperparam_reconstruction_pca_unlabeled
JOBNAME=job_anomaly_detection_hyperparam_reconstruction_pca_unlabeled_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBNAME}
gsutil -m rm -rf ${OUTDIR}
gcloud ml-engine jobs submit training ${JOBNAME} \
  --region=${REGION} \
  --module-name=trainer.task \
  --package-path=$PWD/anomaly_detection_module/trainer \
  --job-dir=${OUTDIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=STANDARD_1 \
  --config=hyperparam_reconstruction_pca.yaml \
  --runtime-version=${TFVERSION} \
  -- \
  --train_file_pattern=gs://${BUCKET}/anomaly_detection/data/train_norm_seq.csv \
  --eval_file_pattern=gs://${BUCKET}/anomaly_detection/data/val_norm_1_seq.csv \
  --output_dir=${OUTDIR} \
  --job-dir=${OUTDIR} \
  --seq_len=30 \
  --num_feat=5 \
  --feat_names=${FEAT_NAMES} \
  --feat_defaults=${FEAT_DEFAULTS} \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --previous_train_steps=0 \
  --reconstruction_epochs=1.0 \
  --train_examples=64000 \
  --eval_examples=6400 \
  --start_delay_secs=60 \
  --throttle_secs=120 \
  --training_mode="reconstruction" \
  --labeled_tune_thresh=True \
  --num_time_anom_thresh=300 \
  --num_feat_anom_thresh=300

## Train error distribution variables

In [ ]:
# Import os environment variables for error dist training hyperparameters
os.environ["TRAIN_FILE_PATTERN"] = "gs://{}/anomaly_detection/data/val_norm_1_seq.csv".format(BUCKET)
os.environ["EVAL_FILE_PATTERN"] = "gs://{}/anomaly_detection/data/val_norm_1_seq.csv".format(BUCKET)
os.environ["PREVIOUS_TRAIN_STEPS"] = str(2000)
os.environ["TRAIN_EXAMPLES"] = str(6400)
os.environ["TRAINING_MODE"] = "calculate_error_distribution_statistics"
os.environ["EPS"] = "1e-12"

### Dense Autoencoder

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/anomaly_detection/trained_model/dense_unlabeled
JOBNAME=job_anomaly_detection_calculate_error_distribution_statistics_dense_unlabeled_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBNAME}
gcloud ml-engine jobs submit training ${JOBNAME} \
  --region=${REGION} \
  --module-name=trainer.task \
  --package-path=$PWD/anomaly_detection_module/trainer \
  --job-dir=${OUTDIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=STANDARD_1 \
  --runtime-version=${TFVERSION} \
  -- \
  --train_file_pattern=${TRAIN_FILE_PATTERN} \
  --eval_file_pattern=${EVAL_FILE_PATTERN} \
  --output_dir=${OUTDIR} \
  --job-dir=./tmp \
  --seq_len=${SEQ_LEN} \
  --num_feat=${NUM_FEAT} \
  --feat_names=${FEAT_NAMES} \
  --feat_defaults=${FEAT_DEFAULTS} \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --previous_train_steps=${PREVIOUS_TRAIN_STEPS} \
  --train_examples=${TRAIN_EXAMPLES} \
  --start_delay_secs=${START_DELAY_SECS} \
  --throttle_secs=${THROTTLE_SECS} \
  --model_type="dense_autoencoder" \
  --enc_dnn_hidden_units=${ENC_DNN_HIDDEN_UNITS} \
  --latent_vector_size=${LATENT_VECTOR_SIZE} \
  --dec_dnn_hidden_units=${DEC_DNN_HIDDEN_UNITS} \
  --time_loss_weight=${TIME_LOSS_WEIGHT} \
  --feat_loss_weight=${FEAT_LOSS_WEIGHT} \
  --training_mode=${TRAINING_MODE} \
  --labeled_tune_thresh=${LABELED_TUNE_THRESH} \
  --eps=${EPS}

### LSTM Autoencoder

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/anomaly_detection/trained_model/lstm_unlabeled
JOBNAME=job_anomaly_detection_calculate_error_distribution_statistics_lstm_unlabeled_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBNAME}
gcloud ml-engine jobs submit training ${JOBNAME} \
  --region=${REGION} \
  --module-name=trainer.task \
  --package-path=$PWD/anomaly_detection_module/trainer \
  --job-dir=${OUTDIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=STANDARD_1 \
  --runtime-version=${TFVERSION} \
  -- \
  --train_file_pattern=${TRAIN_FILE_PATTERN} \
  --eval_file_pattern=${EVAL_FILE_PATTERN} \
  --output_dir=${OUTDIR} \
  --job-dir=./tmp \
  --seq_len=${SEQ_LEN} \
  --num_feat=${NUM_FEAT} \
  --feat_names=${FEAT_NAMES} \
  --feat_defaults=${FEAT_DEFAULTS} \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --previous_train_steps=${PREVIOUS_TRAIN_STEPS} \
  --train_examples=${TRAIN_EXAMPLES} \
  --start_delay_secs=${START_DELAY_SECS} \
  --throttle_secs=${THROTTLE_SECS} \
  --model_type="lstm_enc_dec_autoencoder" \
  --reverse_labels_sequence=${REVERSE_LABELS_SEQUENCE} \
  --enc_lstm_hidden_units=${ENC_LSTM_HIDDEN_UNITS} \
  --dec_lstm_hidden_units=${DEC_LSTM_HIDDEN_UNITS} \
  --lstm_dropout_output_keep_probs=${LSTM_DROPOUT_OUTPUT_KEEP_PROBS} \
  --dnn_hidden_units=${DNN_HIDDEN_UNITS} \
  --training_mode=${TRAINING_MODE} \
  --labeled_tune_thresh=${LABELED_TUNE_THRESH} \
  --eps=${EPS}

### PCA Autoencoder

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/anomaly_detection/trained_model/pca_unlabeled
JOBNAME=job_anomaly_detection_calculate_error_distribution_statistics_pca_unlabeled_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBNAME}
gcloud ml-engine jobs submit training ${JOBNAME} \
  --region=${REGION} \
  --module-name=trainer.task \
  --package-path=$PWD/anomaly_detection_module/trainer \
  --job-dir=${OUTDIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=STANDARD_1 \
  --runtime-version=${TFVERSION} \
  -- \
  --train_file_pattern=${TRAIN_FILE_PATTERN} \
  --eval_file_pattern=${EVAL_FILE_PATTERN} \
  --output_dir=${OUTDIR} \
  --job-dir=./tmp \
  --seq_len=${SEQ_LEN} \
  --num_feat=${NUM_FEAT} \
  --feat_names=${FEAT_NAMES} \
  --feat_defaults=${FEAT_DEFAULTS} \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --previous_train_steps=2200 \
  --train_examples=${TRAIN_EXAMPLES} \
  --start_delay_secs=${START_DELAY_SECS} \
  --throttle_secs=${THROTTLE_SECS} \
  --model_type="pca" \
  --training_mode=${TRAINING_MODE} \
  --labeled_tune_thresh=${LABELED_TUNE_THRESH} \
  --eps=${EPS}

## Tune anomaly thresholds

In [ ]:
# Import os environment variables for tune threshold training hyperparameters
os.environ["PREVIOUS_TRAIN_STEPS"] = str(2200)
os.environ["TRAIN_EXAMPLES"] = str(12800)
os.environ["TRAINING_MODE"] = "tune_anomaly_thresholds"

## Unlabeled

In [ ]:
# Import os environment variables for unlabeled tune threshold training hyperparameters
os.environ["TRAIN_FILE_PATTERN"] = "gs://{}/anomaly_detection/data/unlabeled_val_mixed_seq.csv".format(BUCKET)
os.environ["EVAL_FILE_PATTERN"] = "gs://{}/anomaly_detection/data/unlabeled_val_mixed_seq.csv".format(BUCKET)
os.environ["TIME_THRESH_SCL"] = str(2.0)
os.environ["FEAT_THRESH_SCL"] = str(2.0)

### Dense Autoencoder

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/anomaly_detection/trained_model/dense_unlabeled
JOBNAME=job_anomaly_detection_tune_anomaly_thresholds_dense_unlabeled_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBNAME}
gcloud ml-engine jobs submit training ${JOBNAME} \
  --region=${REGION} \
  --module-name=trainer.task \
  --package-path=$PWD/anomaly_detection_module/trainer \
  --job-dir=${OUTDIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=STANDARD_1 \
  --runtime-version=${TFVERSION} \
  -- \
  --train_file_pattern=${TRAIN_FILE_PATTERN} \
  --eval_file_pattern=${EVAL_FILE_PATTERN} \
  --output_dir=${OUTDIR} \
  --job-dir=./tmp \
  --seq_len=${SEQ_LEN} \
  --num_feat=${NUM_FEAT} \
  --feat_names=${FEAT_NAMES} \
  --feat_defaults=${FEAT_DEFAULTS} \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --previous_train_steps=${PREVIOUS_TRAIN_STEPS} \
  --train_examples=${TRAIN_EXAMPLES} \
  --start_delay_secs=${START_DELAY_SECS} \
  --throttle_secs=${THROTTLE_SECS} \
  --model_type="dense_autoencoder" \
  --enc_dnn_hidden_units=${ENC_DNN_HIDDEN_UNITS} \
  --latent_vector_size=${LATENT_VECTOR_SIZE} \
  --dec_dnn_hidden_units=${DEC_DNN_HIDDEN_UNITS} \
  --time_loss_weight=${TIME_LOSS_WEIGHT} \
  --feat_loss_weight=${FEAT_LOSS_WEIGHT} \
  --training_mode=${TRAINING_MODE} \
  --labeled_tune_thresh=${LABELED_TUNE_THRESH} \
  --time_thresh_scl=${TIME_THRESH_SCL} \
  --feat_thresh_scl=${FEAT_THRESH_SCL}

### LSTM Autoencoder

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/anomaly_detection/trained_model/lstm_unlabeled
JOBNAME=job_anomaly_detection_tune_anomaly_thresholds_lstm_unlabeled_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBNAME}
gcloud ml-engine jobs submit training ${JOBNAME} \
  --region=${REGION} \
  --module-name=trainer.task \
  --package-path=$PWD/anomaly_detection_module/trainer \
  --job-dir=${OUTDIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=STANDARD_1 \
  --runtime-version=${TFVERSION} \
  -- \
  --train_file_pattern=${TRAIN_FILE_PATTERN} \
  --eval_file_pattern=${EVAL_FILE_PATTERN} \
  --output_dir=${OUTDIR} \
  --job-dir=./tmp \
  --seq_len=${SEQ_LEN} \
  --num_feat=${NUM_FEAT} \
  --feat_names=${FEAT_NAMES} \
  --feat_defaults=${FEAT_DEFAULTS} \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --previous_train_steps=${PREVIOUS_TRAIN_STEPS} \
  --train_examples=${TRAIN_EXAMPLES} \
  --start_delay_secs=${START_DELAY_SECS} \
  --throttle_secs=${THROTTLE_SECS} \
  --model_type="lstm_enc_dec_autoencoder" \
  --reverse_labels_sequence=${REVERSE_LABELS_SEQUENCE} \
  --enc_lstm_hidden_units=${ENC_LSTM_HIDDEN_UNITS} \
  --dec_lstm_hidden_units=${DEC_LSTM_HIDDEN_UNITS} \
  --lstm_dropout_output_keep_probs=${LSTM_DROPOUT_OUTPUT_KEEP_PROBS} \
  --dnn_hidden_units=${DNN_HIDDEN_UNITS} \
  --training_mode=${TRAINING_MODE} \
  --labeled_tune_thresh=${LABELED_TUNE_THRESH} \
  --time_thresh_scl=${TIME_THRESH_SCL} \
  --feat_thresh_scl=${FEAT_THRESH_SCL}

### PCA Autoencoder

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/anomaly_detection/trained_model/pca_unlabeled
JOBNAME=job_anomaly_detection_tune_anomaly_thresholds_pca_unlabeled_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBNAME}
gcloud ml-engine jobs submit training ${JOBNAME} \
  --region=${REGION} \
  --module-name=trainer.task \
  --package-path=$PWD/anomaly_detection_module/trainer \
  --job-dir=${OUTDIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=STANDARD_1 \
  --runtime-version=${TFVERSION} \
  -- \
  --train_file_pattern=${TRAIN_FILE_PATTERN} \
  --eval_file_pattern=${EVAL_FILE_PATTERN} \
  --output_dir=${OUTDIR} \
  --job-dir=./tmp \
  --seq_len=${SEQ_LEN} \
  --num_feat=${NUM_FEAT} \
  --feat_names=${FEAT_NAMES} \
  --feat_defaults=${FEAT_DEFAULTS} \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --previous_train_steps=2400 \
  --train_examples=${TRAIN_EXAMPLES} \
  --start_delay_secs=${START_DELAY_SECS} \
  --throttle_secs=${THROTTLE_SECS} \
  --model_type="pca" \
  --training_mode=${TRAINING_MODE} \
  --labeled_tune_thresh=${LABELED_TUNE_THRESH} \
  --time_thresh_scl=${TIME_THRESH_SCL} \
  --feat_thresh_scl=${FEAT_THRESH_SCL}

## Deploy

### Dense Autoencoder

In [ ]:
%%bash
MODEL_NAME="anomaly_detection_dense_unlabeled"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/anomaly_detection/trained_model/dense_unlabeled/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions ${REGION}
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version ${TFVERSION}

### LSTM Autoencoder

In [ ]:
%%bash
MODEL_NAME="anomaly_detection_lstm_unlabeled"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/anomaly_detection/trained_model/lstm_unlabeled/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions ${REGION}
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version ${TFVERSION}

### PCA Autoencoder

In [ ]:
%%bash
MODEL_NAME="anomaly_detection_pca_unlabeled"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/anomaly_detection/trained_model/pca_unlabeled/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions ${REGION}
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version ${TFVERSION}

## Prediction

In [29]:
UNLABELED_CSV_COLUMNS = ["tag_{0}".format(tag) for tag in range(0, 5)]

In [30]:
import numpy as np
unlabeled_test_mixed_sequences_array = np.loadtxt(
    fname="data/unlabeled_test_mixed_seq.csv", dtype=str, delimiter=",")
print("unlabeled_test_mixed_sequences_array.shape = {}".format(
    unlabeled_test_mixed_sequences_array.shape))

unlabeled_test_mixed_sequences_array.shape = (12800, 5)


In [31]:
number_of_prediction_instances = 10
print("labels = {}".format(
  unlabeled_test_mixed_sequences_array[0:number_of_prediction_instances, -1]))

labels = ['0.10912429;1.21779374;1.52916082;-0.177806;-0.54337799;0.44055714;1.27923318;1.18207574;-0.84995686;0.08906497;0.65940998;1.61871863;0.81842511;-0.96566587;-0.60382206;1.69003285;1.66798859;0.31696774;-0.408903;-0.25624363;1.39733346;1.38432466;-0.19920215;-0.79272972;0.38229627;1.13707338;1.69781059;0.19309917;-0.97771496;0.44386982'
 '0.14320316;1.29770315;1.07947085;-0.1310581;-0.3940327;1.23110166;1.25224647;0.83662982;-0.94877962;-0.00280088;0.60816257;1.16871775;0.67355164;-0.33326375;-0.16736942;0.78481252;1.17160361;0.34967432;-0.97550557;0.52889076;1.63690058;1.67796765;-0.1170558;-0.95527216;0.50201129;1.89823695;1.12541774;0.27771255;-0.11889077;0.54388248'
 '0.92380376;1.1275517;0.68680086;-0.65989216;-0.65966357;1.13504582;1.71752066;1.16837965;-0.71586137;-0.13441075;1.14169827;1.99655389;0.69922314;-0.6801096;0.29903466;1.68053185;1.2336307;0.32646373;-1.03204426;-0.33344275;1.11825307;26.42256512;-7.4768817;6.55099231;-1.56550118;23.40616989;-12.96872267;0.55

### GCloud ML-Engine prediction from deployed model

In [32]:
test_data_normal_string_list = unlabeled_test_mixed_sequences_array.tolist()[0:number_of_prediction_instances]

In [33]:
# Format dataframe to instances list to get sent to ML-Engine
instances = [{UNLABELED_CSV_COLUMNS[i]: example[i]
              for i in range(len(UNLABELED_CSV_COLUMNS))} 
             for example in unlabeled_test_mixed_sequences_array.tolist()[0:number_of_prediction_instances]]

In [34]:
instances

[{'tag_0': '0.08630851;2.37045866;1.20743555;-1.14710842;-0.48609804;1.50719389;2.19601683;0.10701992;-1.24018965;-0.19057113;1.8352755;1.752288;-0.68841143;-1.25716387;0.38385502;2.6536265;0.79187743;-1.04243811;-0.52800537;1.76135306;2.42560225;0.47596221;-0.93880634;0.34620397;1.77331762;1.74868177;-0.03435227;-0.88146609;0.34844452;2.28759207',
  'tag_1': '0.01710703;1.43010645;-0.40862645;0.54163604;2.06510263;-0.00407659;-0.12560559;1.91115068;0.08341453;-0.99214653;1.59915954;0.90426161;-1.14516381;0.92210796;0.52287498;-0.74294574;0.70845168;1.03833225;-0.4001544;0.95978758;1.08263277;-0.31584913;0.00862613;1.30877146;-0.22048323;-0.24728125;1.4809888;-0.20183649;-0.16185678;1.24482918',
  'tag_2': '0.94729535;2.04044194;1.47843099;0.16625841;-0.75320741;-0.41766578;1.258229;1.52247393;0.72566737;-0.61741024;-0.55178713;0.43867216;1.09453834;1.59920523;0.34086948;-0.8572519;-0.9924474;0.89896643;1.69688929;1.0605381;-0.45109315;-0.96045705;-0.59540668;1.24977148;2.10291277;0.88

### Dense Autoencoder

In [35]:
# Send instance dictionary to receive response from ML-Engine for online prediction
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build("ml", "v1", credentials = credentials)

request_data = {"instances": instances}

parent = "projects/%s/models/%s/versions/%s" % (PROJECT, "anomaly_detection_dense_unlabeled", "v1")
response = api.projects().predict(body = request_data, name = parent).execute()
print("response = {}".format(response))

response = {'predictions': [{'feat_anom_flags': 0, 'X_time_abs_recon_err': [[0.08630851, 0.01710703, 0.94729535, 0.39874107, 0.10912429], [2.37045866, 1.43010645, 2.04044194, 2.00116413, 1.21779374], [1.20743555, 0.40862645, 1.47843099, 0.31937228, 1.52916082], [1.14710842, 0.54163604, 0.16625841, 0.16407445, 0.177806], [0.48609804, 2.06510263, 0.75320741, 1.40468418, 0.54337799], [1.50719389, 0.00407659, 0.41766578, 0.85744544, 0.44055714], [2.19601683, 0.12560559, 1.258229, 0.96783301, 1.27923318], [0.10701992, 1.91115068, 1.52247393, 0.09800176, 1.18207574], [1.24018965, 0.08341453, 0.72566737, 1.56246727, 0.84995686], [0.19057113, 0.99214653, 0.61741024, 0.24066002, 0.08906497], [1.8352755, 1.59915954, 0.55178713, 0.20638952, 0.65940998], [1.752288, 0.90426161, 0.43867216, 1.49479056, 1.61871863], [0.68841143, 1.14516381, 1.09453834, 0.8453935, 0.81842511], [1.25716387, 0.92210796, 1.59920523, 0.65794037, 0.96566587], [0.38385502, 0.52287498, 0.34086948, 0.81768388, 0.60382206], [2

### LSTM Autoencoder

In [36]:
# Send instance dictionary to receive response from ML-Engine for online prediction
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build("ml", "v1", credentials = credentials)

request_data = {"instances": instances}

parent = "projects/%s/models/%s/versions/%s" % (PROJECT, "anomaly_detection_lstm_unlabeled", "v1")
response = api.projects().predict(body = request_data, name = parent).execute()
print("response = {}".format(response))

response = {'predictions': [{'feat_anom_flags': 0, 'X_time_abs_recon_err': [[0.4291785439097315, 0.3837780556630957, 1.2928434014560843, 0.7440698048830452, 0.4432281861612492], [2.7133286939097316, 1.7967774756630956, 2.3859899914560843, 2.346492864883045, 1.5518976361612493], [1.5503055839097315, 0.0419554243369043, 1.8239790414560841, 0.6647010148830452, 1.8632647161612492], [0.8042383860902684, 0.9083070656630956, 0.5118064614560843, 0.1812542848830452, 0.1562978961612492], [0.14322800609026853, 2.431773655663096, 0.4076593585439157, 1.7500129148830452, 0.20927409383875079], [1.8500639239097314, 0.3625944356630957, 0.07211772854391568, 1.2027741748830452, 0.7746610361612492], [2.5388868639097315, 0.24106543566309568, 1.6037770514560843, 0.6225042751169547, 1.6133370761612493], [0.44988995390973147, 2.277821705663096, 1.8680219814560843, 0.4433304948830452, 1.5161796361612492], [0.8973196160902686, 0.45008555566309566, 1.0712154214560843, 1.9077960048830451, 0.5158529638387508], [0.

### PCA Autoencoder

In [37]:
# Send instance dictionary to receive response from ML-Engine for online prediction
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build("ml", "v1", credentials = credentials)

request_data = {"instances": instances}

parent = "projects/%s/models/%s/versions/%s" % (PROJECT, "anomaly_detection_pca_unlabeled", "v1")
response = api.projects().predict(body = request_data, name = parent).execute()
print("response = {}".format(response))

response = {'predictions': [{'feat_anom_flags': 0, 'X_time_abs_recon_err': [[0.05855919433258866, 0.49859375222010854, 0.4402078430191428, 0.09126535727640259, 0.12279726587824286], [0.10734155391594857, 0.859007319121039, 1.3976481472715585, 1.2460769911934573, 0.2470381787927628], [0.2762861227137875, 0.9572483704425702, 0.8906983825981, 0.3281615638088393, 0.5645673802834239], [0.015942816211133914, 0.052139600236564686, 0.2766377614746027, 0.5286932815852369, 0.17345959952139434], [0.14818932165640408, 1.5643712260493599, 1.2236252549046478, 0.9863061232237782, 0.4421505975500568], [0.24870158607659076, 0.5476058264051152, 0.9929231196016819, 0.23428005375223346, 0.4106967674591592], [0.14647718361356232, 0.6870212778722063, 0.6391562046570125, 1.6765849416309777, 0.02990980109603847], [0.39592098401335163, 1.3847052927148453, 0.9890659553427278, 0.44341752749164415, 0.7110290973489316], [0.06747709312620076, 0.40209102465052426, 0.2925474501399289, 1.2169447958148756, 0.4098726633